_PDF_ táblázatok `pandas`-ba való alakítása. Olyan _PDF_-ekre, amelyek képekből vannak - tehát fényképek, szkennelések, vagy hasonló. Ez magában foglalja a sima fényképek (_JPG_, _PNG_) szövegfelismerését is. Az átalakítási folyamat három lépéses:
 1. _PDF_ oldalainak képekké alakítása
 2. Szövegfelismerés a képeken (ezt angolul úgy hívják _OCR - optical character recognition_)
 3. Szöveg táblázattá alakítása

### Installs

#### ImageMagick és GhostSCript

Kell hozzá `ImageMagick` és `GhostScript`.

https://www.imagemagick.org/script/download.php

https://www.ghostscript.com/download/gsdnld.html

#### Poppler

Install Poppler, add to PATH

Go to [this](http://blog.alivate.com.au/poppler-windows/) page and download the binary of your choice. In this example we will download and use poppler-0.68.0. Extract the archive file poppler-0.68.0_x86.7z into C:\Program Files\poppler. Thus the directory structure should look something like this:

C:  
  └ Program Files  
    └ poppler  
    └ poppler-0.68.0  
      └ bin  
      └ include  
      └ lib  
      └ share  

Windows: Add `C:\Program Files\poppler\poppler-0.68.0\bin` to your system PATH.

Valamit így kell a PATH-hoz hozzáadni: https://www.architectryan.com/2018/03/17/add-to-the-path-on-windows-10/

#### Tesseract

Install Tesseract: https://github.com/tesseract-ocr/tesseract/wiki  
For Windows: https://github.com/UB-Mannheim/tesseract/wiki

Add `C:\Program Files\Tesseract-OCR` to your system PATH.

#### Restart Python

Ezután indítsd újra az Anacondát és a JupyterLabet és a Python-t.

Olvasd el: https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/

In [1]:
!pip install Pillow

In [2]:
!pip install pdf2image

Olvasd el: https://pypi.org/project/pytesseract/

In [3]:
!pip install pytesseract

Olvasd el: https://stackoverflow.com/questions/44439443/python-how-to-pip-install-opencv2-with-specific-version-2-4-9

In [4]:
!pip install opencv-python

# 1

https://hoineki.com/article.php?a=How_to_convert_PDF_to_Image34

In [5]:
from PIL import Image 
import sys 
from pdf2image import convert_from_path 
import os 

In [6]:
# Path of the pdf 
PDF_file = "Lista inscrisi Admitere Licenta sept 11.09.2015.pdf"

In [7]:
if not os.path.exists('pdf/'+PDF_file+'/'):
    os.makedirs('pdf/'+PDF_file+'/')

Készítünk egy mappát, ahová a _PDF_ oldalait exportáljuk képként.

In [40]:
# Store all the pages of the PDF in a variable 
#a masodik szam a felbontas, ezt 300-600 kozott probalgasd
pages = convert_from_path(PDF_file, 300) 
  
# Counter to store images of each page of PDF to image 
image_counter = 1
  
# Iterate through all the pages stored above 
for page in pages: 
  
    # Declaring filename for each page of PDF as JPG 
    # For each page, filename will be: 
    # PDF page 1 -> page_1.jpg 
    # PDF page 2 -> page_2.jpg 
    # PDF page 3 -> page_3.jpg 
    # .... 
    # PDF page n -> page_n.jpg 
    filename = 'pdf/'+PDF_file+"/page_"+str(image_counter)+".jpg"
    print(image_counter,'oldal kész..')
      
    # Save the image of the page in system 
    page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
    image_counter = image_counter + 1

1 oldal kész..
2 oldal kész..
3 oldal kész..
4 oldal kész..
5 oldal kész..


# 2

Szövegfelismerés a képeken

In [9]:
import pytesseract

In [47]:
# Variable to get count of total number of pages 
filelimit = image_counter-1
  
# Creating a text file to write the output 
outfile = 'pdf/'+PDF_file+"/text.txt"
  
# Open the file in append mode so that  
# All contents of all images are added to the same file 
f = open(outfile, "a") 
  
# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 
  
    # Set filename to recognize text from 
    # Again, these files will be: 
    # page_1.jpg 
    # page_2.jpg 
    # .... 
    # page_n.jpg 
    filename = 'pdf/'+PDF_file+"/page_"+str(i)+".jpg"
          
    # Recognize the text as string in image using pytesserct 
    text = str(((pytesseract.image_to_string(Image.open(filename))))) 
    print(i,'oldal kész..')
  
    # The recognized text is stored in variable text 
    # Any string processing may be applied on text 
    # Here, basic formatting has been done: 
    # In many PDFs, at line ending, if a word can't 
    # be written fully, a 'hyphen' is added. 
    # The rest of the word is written in the next line 
    # Eg: This is a sample text this word here GeeksF- 
    # orGeeks is half on first line, remaining on next. 
    # To remove this, we replace every '-\n' to ''. 
    text = text.replace('-\n', '')     
  
    # Finally, write the processed text to the file. 
    f.write(text) 

# Close the file after writing all the text. 
f.close() 

1 oldal kész..
2 oldal kész..
3 oldal kész..
4 oldal kész..
5 oldal kész..


# 3

In [48]:
import pandas as pd

Felismert szöveg beolvasása

In [325]:
pages=open(outfile,'r').read()

Sorok felosztása újsork karakterek `\n` szerint

In [326]:
lines=[i for i in pages.split('\n') if i]

Csak azokat a sorokat tartjuk meg, amelyek számmal kezdődnek

In [327]:
good_lines=[line for line in lines if line[0].isdigit()]

Tipikus hibás felismert karaterek javítása

In [328]:
good_lines=[line.replace('_',' ').replace('. ',' ').replace('-',' ').replace('—',' ')\
     .replace('~',' ').replace('=',' ').replace('  ',' ').replace('  ',' ')\
     .replace('»',' ')for line in good_lines]

In [329]:
good_lines[:10]

['1 4696 ACHIM O.A ANDREEA MAGDALENA 9,56 9,50 9,70 19 18 16 14 13 11 15 20 12',
 '3 4441 ALBERT J CSILLA EDIT 6,76 5,70 7,80 48 ,',
 '2 208 ALBERT Vv IULAN 6,10 5,20 600 51.',
 '4 4500 ALEXAN A MARIOARA 7,82 7,00 855 41',
 '5 4565 AMBRUS T TIBOR 8,36 9,10 940 46.',
 '6 4424 ANDREI C IOAN CLAUDIU 7,66 7,00 6,80 19.',
 '7 210 ASANOV A REFAT 7,20 7,20 7,20 36 37',
 '8 4913 BACIU I IOAN ALEXANDRU 7,11 6,60 7,55 59 61',
 '9 4509 BALANESCU D.M RAZVAN SAUL 7,75 9,85 545 12 13',
 '10 4962 BALAS I MARIA ELENA 7,73 7,85 7,60 70.']

Hibás sorok további kiszűrése manuálisan

In [330]:
for i,l in enumerate(good_lines):
    print(str(i)+':::'+l)

0:::1 4696 ACHIM O.A ANDREEA MAGDALENA 9,56 9,50 9,70 19 18 16 14 13 11 15 20 12
1:::3 4441 ALBERT J CSILLA EDIT 6,76 5,70 7,80 48 ,
2:::2 208 ALBERT Vv IULAN 6,10 5,20 600 51.
3:::4 4500 ALEXAN A MARIOARA 7,82 7,00 855 41
4:::5 4565 AMBRUS T TIBOR 8,36 9,10 940 46.
5:::6 4424 ANDREI C IOAN CLAUDIU 7,66 7,00 6,80 19.
6:::7 210 ASANOV A REFAT 7,20 7,20 7,20 36 37
7:::8 4913 BACIU I IOAN ALEXANDRU 7,11 6,60 7,55 59 61
8:::9 4509 BALANESCU D.M RAZVAN SAUL 7,75 9,85 545 12 13
9:::10 4962 BALAS I MARIA ELENA 7,73 7,85 7,60 70.
10:::11 44114 BALINT c ROBERT ISTVAN 7,93 6,80 9,00 36 37,16 18
11:::12 4571 BALOG | MARIAN BOGDAN 6,80 5,25 6,10 46.
12:::13 4526 BANDICI .M RAZVAN ADRIAN 8,93 5,80 815 46.
13:::15 4902 BARABAS L SZENDE 7,01 7,05 7,60 64 59 67
14:::14 4572 BARABAS O BENIAMIN OVIDIU 8,10 650 8,60 11 16 14 15
15:::16 4508 BARATI Vv NINEL EMIL 6,95 6,30 7,50 47.46 41
16:::17 4695 BARDAN Vv SIMINA 9,30 8,70 9,20 37 18 27 36 16 19
17:::18 4669 BARSAN S MARIA ANCA 7,65 6,30 8,30 41 44 46
1

In [331]:
good_lines[126]=good_lines[126].split(',ve Nr Leg')[0]

In [332]:
good_lines[146]=good_lines[146].replace('7 AL','7,41')

In [333]:
helper=good_lines[195].split('199 4514')
good_lines[195]=helper[0]
good_lines.append('199 4514'+helper[1])

Adatok feldolgozása

A név nem mindenhol ugyanolyan hosszú, ezért ezt normalizáljuk.  
A név második neve, az iniciálő sokszor `1` vagy `|`, `l`, `I`. Ezt normalizáljuk I-re.

In [334]:
clean_lines=[]
for line in good_lines:
    values=line.split(' ')
    
    try:
        for k in [2,3,4]:
            values[k]=values[k].replace('1','I').replace('l','I')\
                .replace('|','I').replace('7','I').replace('0','O')

        counter=0
        name_start=100
        name_end=0
        while counter<len(values)-1:
            counter+=1
            try:
                if not values[counter][0].isdigit():
                    name_start=min(name_start,counter)
                    if values[counter+1][0].isdigit():
                        name_end=counter+1
                        counter=len(values)
            except:
                print('HIBA: '+line)
    
        clean_values=values[:name_start]+[' '.join(values[name_start:name_end])]+values[name_end:]
        clean_values=[szo for szo in clean_values if szo]
        clean_lines.append(clean_values)

    except:
        print('HIBA: '+line)

HIBA: 4 Bp Sp


In [341]:
df=pd.DataFrame(clean_lines)
df=df[range(6)]
df.columns=['ID','Leg','Nume','Admitere','Bac 2','Lb. Mat.']
df.head()

,ID,Leg,Nume,Admitere,Bac 2,Lb. Mat.
0,1,4696,ACHIM O.A ANDREEA MAGDALENA,9.56,9.5,9.70
1,3,4441,ALBERT J CSILLA EDIT,6.76,5.7,7.80
2,2,208,ALBERT Vv IULAN,6.10,5.2,6.00
3,4,4500,ALEXAN A MARIOARA,7.82,7.0,8.55
4,5,4565,AMBRUS T TIBOR,8.36,9.1,9.40


In [350]:
df.to_excel('pdf/'+PDF_file+'/data.xlsx')

### extra

Ez a további rész további tisztítás, ezt végezhetjük már akár Excelben is.

A jegyeknél vagy ahol hiányzik a vessző. Ezt úgy vizsgáljuk, hogy a az első karater után mindig vesszőt teszük, ha már nincs. Aztán számokká konvertáljuk.

In [340]:
for line in clean_lines:
    try:
        for i in [3,4,5]:
            jegy=line[i]
            jegy=jegy.replace('.',',')
            jegy=jegy.replace('I','1').replace('l','1')\
                .replace('|','1').replace('A','4').replace('O','0')

            if jegy[0]==',':jegy=jegy[1:]
            if jegy[-1]==',':jegy=jegy[:-1]
            if jegy!='10,00':
                if jegy!='1000':
                    if len(jegy)<4:
                        jegy=jegy[0]+','+jegy[1:]
                    jegy=float(jegy.replace(',','.'))
                else: jegy=10
            else: jegy=10
            line[i]=jegy
    except:
        print(line)

In [342]:
df=pd.DataFrame(clean_lines)
df=df[range(6)]
df.columns=['ID','Leg','Nume','Admitere','Bac 2','Lb. Mat.']
df.head()

,ID,Leg,Nume,Admitere,Bac 2,Lb. Mat.
0,1,4696,ACHIM O.A ANDREEA MAGDALENA,9.56,9.5,9.70
1,3,4441,ALBERT J CSILLA EDIT,6.76,5.7,7.80
2,2,208,ALBERT Vv IULAN,6.10,5.2,6.00
3,4,4500,ALEXAN A MARIOARA,7.82,7.0,8.55
4,5,4565,AMBRUS T TIBOR,8.36,9.1,9.40


In [349]:
df['Mean']=df.mean(axis=1)
df

,ID,Leg,Nume,Admitere,Bac 2,Lb. Mat.,Mean
0,1,4696,ACHIM O.A ANDREEA MAGDALENA,9.56,9.50,9.70,9.586667
1,3,4441,ALBERT J CSILLA EDIT,6.76,5.70,7.80,6.753333
2,2,208,ALBERT Vv IULAN,6.10,5.20,6.00,5.766667
3,4,4500,ALEXAN A MARIOARA,7.82,7.00,8.55,7.790000
4,5,4565,AMBRUS T TIBOR,8.36,9.10,9.40,8.953333
...,...,...,...,...,...,...,...
303,308,4425,ZEGREAN D ANA MARIA,7.51,6.60,8.00,7.370000
304,309,7908,"ZSIGA M KAROLA ,",7.32,6.95,8.40,7.556667
305,310,4410,"ZSURZSAN G. ZS, VIVIEN MERCEDESZ",8.02,7.20,8.65,7.956667
306,311,4562,ZUGRAV I IOANA /,6.58,5.00,6.75,6.110000
